In [1]:
# 기본 라이브러리, 데이터 불러오기

import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

#한글 폰트 설정
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# 데이터 불러오기
df = pd.read_csv("dataset/instax_sales_transaction_data.csv")

print("="*50)
print("데이터셋 기본정보")
print("="*50)
print(f"전체 데이터 개수:{len(df)} 행")
print(f"컬럼 개수: {len(df.columns)} 개")
print("\n 데이터 상위 5개 행:")
print(df.head())

print("\n컬럼명 확인:")
print(df.columns.tolist())

print("\n데이터 타입 확인:")
print(df.dtypes)

print("\n 기술 통계량")
print(df.describe())

데이터셋 기본정보
전체 데이터 개수:12334 행
컬럼 개수: 12 개

 데이터 상위 5개 행:
      Tanggal  Tahun  Bulan    Hari Kategori             Nama_Produk  \
0  2022-05-01   2022      5  Sunday     Film  Instax Mini Film (20s)   
1  2022-05-01   2022      5  Sunday     Film  Instax Mini Film (20s)   
2  2022-05-01   2022      5  Sunday     Film  Instax Mini Film (20s)   
3  2022-05-01   2022      5  Sunday   Kamera          Instax Mini 11   
4  2022-05-01   2022      5  Sunday     Film  Instax Mini Film (20s)   

          Lokasi_Toko          Metode_Bayar  Harga_Satuan  Qty  Diskon_IDR  \
0  Tokopedia Official          Kartu Kredit        185000    1        9250   
1  Tokopedia Official          Kartu Kredit        185000    1           0   
2         Shopee Mall  E-Wallet (Gopay/OVO)        185000    2           0   
3         Shopee Mall              Paylater       1100000    1           0   
4         Shopee Mall  E-Wallet (Gopay/OVO)        185000    2           0   

   Total_Penjualan  
0           175750  
1

In [2]:
# 날짜와 할인금액 정보를 기반으로 판매량 분석

print("\n" + "="*50)
print("데이터 전처리 시작")
print("="*50)


    # 날짜 전처리(문자-> 숫자(월/month)로 변경)
    # 날짜 컬럼을 datetime으로 변환
df['Tanggal'] = pd.to_datetime(df['Tanggal'])

    # 월(month) 정보 추출 / Tanggal을 사용하지 않고 가공된 Month를 사용할거야
df['Month'] = df['Tanggal'].dt.month

print(f"\n날짜 범위: {df['Tanggal'].min()} ~ {df['Tanggal'].max()}")

# 결측치 확인
print("\n 결측치 확인:")
print(df.isnull().sum())

# 결측치가 있다면 처리
if df.isnull().sum().sum() > 0:
    print("\n결측치 처리: 결측치가 있는 행 삭제")
    
    df = df.dropna()
    print(f"처리 후 데이터 개수: {len(df)}행")



데이터 전처리 시작

날짜 범위: 2022-05-01 00:00:00 ~ 2025-05-01 00:00:00

 결측치 확인:
Tanggal            0
Tahun              0
Bulan              0
Hari               0
Kategori           0
Nama_Produk        0
Lokasi_Toko        0
Metode_Bayar       0
Harga_Satuan       0
Qty                0
Diskon_IDR         0
Total_Penjualan    0
Month              0
dtype: int64


In [3]:
print("\n" + "=" * 50)
print("탐색적 테이터 분석 (EDA)")
print("=" * 50)

# 할인 금액 분포 확인
print("\n할인 금액 통계:")
print(df['Diskon_IDR'].describe())

# 판매량 분포 확인
print("\n판매량 통계:")
print(df['Qty'].describe())

# 월별 판매 패턴 확인
print("\n월별 평균 판매량:")
monthly_sales = df.groupby('Month')['Qty'].mean().sort_index()
print(monthly_sales)

# 할인 여부에 따른 판매량 비교
df['할인여부'] = df['Diskon_IDR'] > 0
print("\n할인 여부에 따른 평균 판매량:")
print(df.groupby('할인여부')['Qty'].mean())

# 상관관계분석
print("\n주요 변수 간 상관관계")
corr_matrix = df[['Diskon_IDR', 'Month','Qty']].corr()
print(corr_matrix)


탐색적 테이터 분석 (EDA)

할인 금액 통계:
count    1.233400e+04
mean     1.290899e+04
std      4.618240e+04
min      0.000000e+00
25%      0.000000e+00
50%      0.000000e+00
75%      7.500000e+03
max      1.160000e+06
Name: Diskon_IDR, dtype: float64

판매량 통계:
count    12334.000000
mean         1.940814
std          1.151353
min          1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max          5.000000
Name: Qty, dtype: float64

월별 평균 판매량:
Month
1     1.936954
2     1.913043
3     1.957627
4     1.899767
5     1.923523
6     1.900850
7     1.888480
8     2.002871
9     2.027027
10    1.968825
11    1.965475
12    1.920761
Name: Qty, dtype: float64

할인 여부에 따른 평균 판매량:
할인여부
False    1.949537
True     1.919611
Name: Qty, dtype: float64

주요 변수 간 상관관계
            Diskon_IDR     Month       Qty
Diskon_IDR    1.000000  0.240668 -0.028509
Month         0.240668  1.000000  0.006815
Qty          -0.028509  0.006815  1.000000


In [4]:
# 원인/지료 features(특성/변수) 모델이 학습할 때 참고하는 원인이나 힌트가 되는 데이터.
    # 결과 target 모델이 최종적으로 예측하고 싶어하는 결과물
    # df : data frame의 약자/ 파이썬 안에서 쓰는 엑셀 표

print("\n" + "=" * 50)
print("모델 학습 준비")
print("=" * 50)    

features = [
    'Diskon_IDR', #할인금액
    'Month' #판매월
]

target = 'Qty' #판매량

# X = features, y = target 분리
X = df[features]
y = df[target]

print(f"\n독립변수: {features}")
print(f"종속변수: {target}")
print(f"학습 데이터 개수: {len(X)} 개")


모델 학습 준비

독립변수: ['Diskon_IDR', 'Month']
종속변수: Qty
학습 데이터 개수: 12334 개


In [5]:
# 학습/test 분리 , 데이터에서 20%만 테스트할거야
# 42이 숫자에 대한 의미는 크게 없으며, 다른 숫자를 적어도 가능함. 범주가 고정된 랜덤방식 사용.
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

print(f"\n학습 데이터: {len(X_train)} 개")
print(f"테스트 데이터: {len(X_test)} 개")


학습 데이터: 9867 개
테스트 데이터: 2467 개


# 아래 셀은 추가학습을 해야겠당


In [6]:
# 스케일링 /숫자가 크더라도 공정하게 데이터를 판단해서 처리할수 있도록 하는 기능 -> 데이터의 영향력을 공평하게 맞춰주는 작업
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("\n데이터 스케일링 완료")
print(f"원본 데이터 예시:\n{X_train.head()}")
print(f"\n스케일링된 데이터 예시:\n{X_train_scaled[:5]}")


데이터 스케일링 완료
원본 데이터 예시:
       Diskon_IDR  Month
1785        18500     12
9468            0     11
286             0      6
10856           0      1
3844            0      7

스케일링된 데이터 예시:
[[ 0.12375218  1.34830925]
 [-0.27757247  1.08987327]
 [-0.27757247 -0.20230663]
 [-0.27757247 -1.49448652]
 [-0.27757247  0.05612935]]


In [7]:
# 모델학습
model = RandomForestRegressor(
    n_estimators=100,
    random_state=42
)

model.fit(X_train_scaled, y_train)

,n_estimators,100
,criterion,'squared_error'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [8]:
# 성능평가
# 데이터 예측하기
y_train_pred = model.predict(X_train_scaled)

y_test_pred = model.predict(X_test_scaled)


# 성능 지표 계산
# RMSE (Root Mean Squared Error) 평균 제곱근 오차
# R² Score: 결정계수(1에 가까울수록 좋음)
# MAE (Mean Absolute Error) :  평균 절대 오차

# 데이터 성능
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)

test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\n[학습데이터 성능]")
print(f"R² Score: {train_r2:.4f}")
print(f"RMSE: {train_rmse:.4f}")
print(f"MAE: {train_mae:.4f}")

print("\n[테스트 데이터 성능]")
print(f"R² Score: {test_r2:.4f}")
print(f"MAE: {test_rmse:.4f}")
print(f"MAE: {test_mae:.4f}")

# 오차율 계산(평균 판매량 대비)
avg_qty = y_test.mean()
error_rate = (test_mae / avg_qty) * 100
print(f"\n평균 오차율: {error_rate:.2f}%")


[학습데이터 성능]
R² Score: 0.2098
RMSE: 1.0216
MAE: 0.7484

[테스트 데이터 성능]
R² Score: 0.1887
MAE: 1.0444
MAE: 0.7639

평균 오차율: 39.08%


In [9]:
# 중요도 분석
print("\n" + "=" * 50)
print("특성 중요도 분석")
print("=" * 50)


feature_importance = pd.DataFrame({
    '특성': features,
    '중요도': model.feature_importances_
}).sort_values('중요도', ascending=False)

print("\n특성별 중요도:")
print(feature_importance)


특성 중요도 분석

특성별 중요도:
           특성       중요도
0  Diskon_IDR  0.907786
1       Month  0.092214


In [10]:
# 예측결과 샘플

print("\n" + "=" *50)
print("예측 결과 샘플(테스트 데이터)")
print("=" * 50)

comparison_df = pd.DataFrame({
    '실제값': y_test[:10].values,
    '예측값': y_test_pred[:10],
    '오차' : np.abs(y_test[:10].values - y_test_pred[:10])
})

comparison_df['예측값'] = comparison_df['예측값'].round(2)
comparison_df['오차'] = comparison_df['오차'].round(2)

print(comparison_df)


예측 결과 샘플(테스트 데이터)
   실제값   예측값    오차
0    1  1.95  0.95
1    4  1.94  2.06
2    1  1.91  0.91
3    2  1.92  0.08
4    1  1.88  0.88
5    4  1.94  2.06
6    1  1.88  0.88
7    1  1.92  0.92
8    2  2.00  0.00
9    1  1.25  0.25


In [11]:
# 피클파일 만들기(모델, 스케일러 저장)

joblib.dump(model, "instax_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("모델저장완료")
print("- instax_model.pkl: 학습된 Random Forest 모델")
print("- scaler.pkl: 데이터 스케일러")

모델저장완료
- instax_model.pkl: 학습된 Random Forest 모델
- scaler.pkl: 데이터 스케일러


print("\n" + "=" * 50)
print("최종 요약")
print("=" * 50)

print(f"""
프로젝트명: 인스탁스 판매량 예측 모델
알고리즘: Random Forest Regressor
학습 데이터: {len(X_train)}개
테스트 데이터: {len(X_test)}개

독립변수:
- 할인금액 (Diskon_IDR)
- 판매월 (Month)

종속변수:
- 판매량 (Qty)

모델 성능 (테스트 데이터):
- R² Score: {test_r2:.4f}
- RMSE: {test_rmse:.4f}
- MAE: {test_mae:.4f}
- 오차율: {error_rate:.2f}%

특성 중요도:
{feature_importance.to_string(index=False)}
""")

# ============================================
# 인스탁스 판매량 예측 모델 개발
# ============================================

# 1. 기본 라이브러리 불러오기
import pandas as pd
import numpy as np
import joblib
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 한글 폰트 설정 (matplotlib)
plt.rcParams['font.family'] = 'Malgun Gothic'  # Windows
# plt.rcParams['font.family'] = 'AppleGothic'  # Mac
plt.rcParams['axes.unicode_minus'] = False

# 그래프 스타일 설정
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# ============================================
# 2. 데이터 로드 및 기본 정보 확인
# ============================================

# 데이터 불러오기
df = pd.read_csv("dataset/instax_sales_transaction_data.csv")

print("=" * 50)
print("데이터셋 기본 정보")
print("=" * 50)
print(f"전체 데이터 개수: {len(df)} 행")
print(f"컬럼 개수: {len(df.columns)} 개")
print("\n데이터 상위 5개 행:")
print(df.head())

print("\n컬럼명 확인:")
print(df.columns.tolist())

print("\n데이터 타입 확인:")
print(df.dtypes)

print("\n기술 통계량:")
print(df.describe())

# ============================================
# 3. 데이터 전처리
# ============================================

print("\n" + "=" * 50)
print("데이터 전처리 시작")
print("=" * 50)

# 3-1. 날짜 컬럼을 datetime으로 변환
df['Tanggal'] = pd.to_datetime(df['Tanggal'])

# 3-2. 월(Month) 정보 추출
df['Month'] = df['Tanggal'].dt.month

print(f"\n날짜 범위: {df['Tanggal'].min()} ~ {df['Tanggal'].max()}")

# 3-3. 결측치 확인
print("\n결측치 확인:")
print(df.isnull().sum())

# 결측치가 있다면 처리
if df.isnull().sum().sum() > 0:
    print("\n결측치 처리: 결측치가 있는 행 삭제")
    df = df.dropna()
    print(f"처리 후 데이터 개수: {len(df)} 행")

# ============================================
# 4. 탐색적 데이터 분석 (EDA)
# ============================================

print("\n" + "=" * 50)
print("탐색적 데이터 분석 (EDA)")
print("=" * 50)

# 4-1. 할인 금액 분포 확인
print("\n할인 금액 통계:")
print(df['Diskon_IDR'].describe())

# 4-2. 판매량 분포 확인
print("\n판매량 통계:")
print(df['Qty'].describe())

# 4-3. 월별 판매 패턴 확인
print("\n월별 평균 판매량:")
monthly_sales = df.groupby('Month')['Qty'].mean().sort_index()
print(monthly_sales)

# 4-4. 할인 여부에 따른 판매량 비교
df['할인여부'] = df['Diskon_IDR'] > 0
print("\n할인 여부에 따른 평균 판매량:")
print(df.groupby('할인여부')['Qty'].mean())

# 4-5. 상관관계 분석
print("\n주요 변수 간 상관관계:")
corr_matrix = df[['Diskon_IDR', 'Month', 'Qty']].corr()
print(corr_matrix)

# ============================================
# 5. 데이터 시각화 - EDA
# ============================================

print("\n데이터 시각화 중...")

# 5-1. 판매량 분포 히스토그램
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.hist(df['Qty'], bins=20, color='skyblue', edgecolor='black')
plt.xlabel('판매량 (Qty)')
plt.ylabel('빈도')
plt.title('판매량 분포')
plt.grid(True, alpha=0.3)

# 5-2. 할인 금액 분포
plt.subplot(1, 2, 2)
plt.hist(df['Diskon_IDR'], bins=30, color='lightcoral', edgecolor='black')
plt.xlabel('할인 금액 (IDR)')
plt.ylabel('빈도')
plt.title('할인 금액 분포')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('eda_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# 5-3. 월별 평균 판매량
plt.figure(figsize=(10, 5))
monthly_sales.plot(kind='bar', color='steelblue')
plt.xlabel('월')
plt.ylabel('평균 판매량')
plt.title('월별 평균 판매량')
plt.xticks(rotation=0)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('monthly_sales.png', dpi=300, bbox_inches='tight')
plt.show()

# 5-4. 할인금액과 판매량의 관계
plt.figure(figsize=(10, 6))
plt.scatter(df['Diskon_IDR'], df['Qty'], alpha=0.5, color='green')
plt.xlabel('할인 금액 (IDR)')
plt.ylabel('판매량 (Qty)')
plt.title('할인 금액과 판매량의 관계')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('discount_vs_qty.png', dpi=300, bbox_inches='tight')
plt.show()

# 5-5. 상관관계 히트맵
plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('변수 간 상관관계')
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

print("그래프 저장 완료!")

# ============================================
# 6. 특성(Feature) 및 타겟(Target) 설정
# ============================================

print("\n" + "=" * 50)
print("모델 학습 준비")
print("=" * 50)

# 독립변수(Features): 할인금액, 판매월
# 종속변수(Target): 판매량
features = [
    'Diskon_IDR',  # 할인금액 (IDR)
    'Month'        # 판매월 (1-12)
]

target = 'Qty'  # 판매량

# X: 독립변수, y: 종속변수 분리
X = df[features]
y = df[target]

print(f"\n독립변수: {features}")
print(f"종속변수: {target}")
print(f"학습 데이터 개수: {len(X)} 개")

# ============================================
# 7. 학습/테스트 데이터 분리
# ============================================

# 데이터를 80% 학습용, 20% 테스트용으로 분리
X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,      # 테스트 데이터 비율: 20%
    random_state=42      # 난수 시드 고정
)

print(f"\n학습 데이터: {len(X_train)} 개")
print(f"테스트 데이터: {len(X_test)} 개")

# ============================================
# 8. 데이터 스케일링 (표준화)
# ============================================

# StandardScaler: 평균 0, 표준편차 1로 데이터 표준화
scaler = StandardScaler()

# 학습 데이터로 스케일러 학습 및 변환
X_train_scaled = scaler.fit_transform(X_train)

# 테스트 데이터는 학습된 스케일러로만 변환
X_test_scaled = scaler.transform(X_test)

print("\n데이터 스케일링 완료")

# ============================================
# 9. 모델 학습 - Random Forest Regressor
# ============================================

print("\n" + "=" * 50)
print("모델 학습 시작")
print("=" * 50)

# Random Forest Regressor 모델 생성
model = RandomForestRegressor(
    n_estimators=100,    # 결정 트리 100개 사용
    random_state=42,     # 난수 시드 고정
    max_depth=10,        # 트리 최대 깊이
    min_samples_split=5, # 노드 분할 최소 샘플 수
    min_samples_leaf=2   # 리프 노드 최소 샘플 수
)

# 모델 학습
model.fit(X_train_scaled, y_train)

print("모델 학습 완료!")
print(f"사용된 트리 개수: {model.n_estimators}")

# ============================================
# 10. 모델 성능 평가
# ============================================

print("\n" + "=" * 50)
print("모델 성능 평가")
print("=" * 50)

# 학습 데이터 예측
y_train_pred = model.predict(X_train_scaled)

# 테스트 데이터 예측
y_test_pred = model.predict(X_test_scaled)

# 성능 지표 계산
# 학습 데이터
train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
train_r2 = r2_score(y_train, y_train_pred)
train_mae = mean_absolute_error(y_train, y_train_pred)

# 테스트 데이터
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))
test_r2 = r2_score(y_test, y_test_pred)
test_mae = mean_absolute_error(y_test, y_test_pred)

print("\n[학습 데이터 성능]")
print(f"R² Score: {train_r2:.4f}")
print(f"RMSE: {train_rmse:.4f}")
print(f"MAE: {train_mae:.4f}")

print("\n[테스트 데이터 성능]")
print(f"R² Score: {test_r2:.4f}")
print(f"RMSE: {test_rmse:.4f}")
print(f"MAE: {test_mae:.4f}")

# 오차율 계산
avg_qty = y_test.mean()
error_rate = (test_mae / avg_qty) * 100
print(f"\n평균 오차율: {error_rate:.2f}%")

# ============================================
# 11. 특성 중요도 분석 및 시각화
# ============================================

print("\n" + "=" * 50)
print("특성 중요도 분석")
print("=" * 50)

# Random Forest 모델의 특성 중요도 추출
feature_importance = pd.DataFrame({
    '특성': features,
    '중요도': model.feature_importances_
}).sort_values('중요도', ascending=False)

print("\n특성별 중요도:")
print(feature_importance)

# 특성 중요도 시각화
plt.figure(figsize=(8, 5))
colors = ['#FF6B6B', '#4ECDC4']
plt.barh(feature_importance['특성'], feature_importance['중요도'], color=colors)
plt.xlabel('중요도')
plt.title('Random Forest 특성 중요도')
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.savefig('feature_importance.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================
# 12. 예측 결과 시각화
# ============================================

print("\n" + "=" * 50)
print("예측 결과 시각화")
print("=" * 50)

# 12-1. 실제값 vs 예측값 산점도
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred, alpha=0.6, color='blue', edgecolor='black')

# 완벽한 예측선 (y=x)
min_val = min(y_test.min(), y_test_pred.min())
max_val = max(y_test.max(), y_test_pred.max())
plt.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='완벽한 예측')

plt.xlabel('실제 판매량')
plt.ylabel('예측 판매량')
plt.title(f'실제값 vs 예측값 (R² = {test_r2:.4f})')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('actual_vs_predicted.png', dpi=300, bbox_inches='tight')
plt.show()

# 12-2. 잔차 플롯 (Residual Plot)
residuals = y_test - y_test_pred

plt.figure(figsize=(10, 6))
plt.scatter(y_test_pred, residuals, alpha=0.6, color='green', edgecolor='black')
plt.axhline(y=0, color='r', linestyle='--', linewidth=2)
plt.xlabel('예측 판매량')
plt.ylabel('잔차 (실제값 - 예측값)')
plt.title('잔차 플롯')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('residual_plot.png', dpi=300, bbox_inches='tight')
plt.show()

# 12-3. 잔차 히스토그램
plt.figure(figsize=(10, 6))
plt.hist(residuals, bins=30, color='orange', edgecolor='black', alpha=0.7)
plt.xlabel('잔차')
plt.ylabel('빈도')
plt.title('잔차 분포')
plt.axvline(x=0, color='r', linestyle='--', linewidth=2)
plt.grid(True, alpha=0.3, axis='y')
plt.tight_layout()
plt.savefig('residual_histogram.png', dpi=300, bbox_inches='tight')
plt.show()

# ============================================
# 13. 예측 결과 샘플 확인
# ============================================

print("\n" + "=" * 50)
print("예측 결과 샘플 (테스트 데이터)")
print("=" * 50)

# 테스트 데이터에서 처음 10개 샘플 비교
comparison_df = pd.DataFrame({
    '실제값': y_test[:10].values,
    '예측값': np.round(y_test_pred[:10], 2),
    '오차': np.round(np.abs(y_test[:10].values - y_test_pred[:10]), 2)
})

print(comparison_df)

# ============================================
# 14. 모델 및 스케일러 저장
# ============================================

print("\n" + "=" * 50)
print("모델 저장")
print("=" * 50)

# 학습된 모델과 스케일러를 피클 파일로 저장
joblib.dump(model, "instax_model.pkl")
joblib.dump(scaler, "scaler.pkl")

print("모델 저장 완료!")
print("- instax_model.pkl: 학습된 Random Forest 모델")
print("- scaler.pkl: 데이터 스케일러")

# ============================================
# 15. 최종 요약 및 보고서용 표
# ============================================

print("\n" + "=" * 50)
print("최종 요약")
print("=" * 50)

# 성능 지표 요약 표
performance_summary = pd.DataFrame({
    '구분': ['학습 데이터', '테스트 데이터'],
    'R² Score': [f"{train_r2:.4f}", f"{test_r2:.4f}"],
    'RMSE': [f"{train_rmse:.4f}", f"{test_rmse:.4f}"],
    'MAE': [f"{train_mae:.4f}", f"{test_mae:.4f}"]
})

print("\n성능 지표 요약:")
print(performance_summary.to_string(index=False))

print(f"""
\n프로젝트 최종 결과
==========================================
알고리즘: Random Forest Regressor
학습 데이터: {len(X_train)}개
테스트 데이터: {len(X_test)}개
트리 개수: {model.n_estimators}

독립변수:
- 할인금액 (Diskon_IDR)
- 판매월 (Month)

종속변수:
- 판매량 (Qty)

모델 성능 (테스트 데이터):
- R² Score: {test_r2:.4f}
- RMSE: {test_rmse:.4f}
- MAE: {test_mae:.4f}
- 오차율: {error_rate:.2f}%

특성 중요도:
{feature_importance.to_string(index=False)}

생성된 그래프 파일:
- eda_distribution.png: 판매량 및 할인금액 분포
- monthly_sales.png: 월별 평균 판매량
- discount_vs_qty.png: 할인금액과 판매량 관계
- correlation_heatmap.png: 변수 간 상관관계
- feature_importance.png: 특성 중요도
- actual_vs_predicted.png: 실제값 vs 예측값
- residual_plot.png: 잔차 플롯
- residual_histogram.png: 잔차 분포
==========================================
""")